# Playground

## Install Dependencies

## Import Dependencies

In [64]:
import numpy as np
import pandas as pd

import random
from random import randint, randrange

## Load train.csv

In [65]:
df_train = pd.read_csv('../data/asl-signs/train.csv')
df_train.shape

(94477, 4)

## Parameters

In [66]:
# random seed used for subsampling
RSEED = 42

# subsamnpling proportion of whole data
p = 0.001
#p = 0.01

## Random subsample

In [67]:
# set random state
random.seed(RSEED)

# set length of subsample
length = int(df_train.shape[0] * p) # number of rows in train.csv times p

# random sampling of row indices
r = [randrange(df_train.shape[0]) for i in range(0, length)] 

# subsampling df_train
df_train_sub = df_train.iloc[r].shape


## Save subsample 

In [70]:
f'../data/train_sub{length}.csv'

'../data/train_sub_94.csv'

In [68]:
df_train_sub.to_csv(f'../data/train_sub{length}.csv', index=False)

(94, 4)